# Analysis of biases of the NC measure

In [ ]:
import numpy as np
from sklearn import datasets
from numpy.linalg import pinv
import matplotlib.pylab as plt

Let's first implement the NC measure as:
$$
\Sigma_w = \frac{1}{n} \sum\limits_{j=1}^K \sum\limits_{y_i = j} (x_i - \mu_j) \cdot (x_i - \mu_j)^T
$$
$$
\Sigma_b = \frac{1}{K} \sum\limits_{j=1}^K (\mu_j - \mu)(\mu_j - \mu)^T
$$
$$
NC = \frac{1}{K} \text{trace}(\Sigma_w \cdot \Sigma_b^{+})
$$
This should match with the definition https://www.pnas.org/doi/full/10.1073/pnas.2015509117.

However, it is unclear whether the within-class covariance $\Sigma_w$, should be rather normalized for each class to compensate
for classes with quite a lot of examples: 
$$
\Sigma_w = \frac{1}{K} \sum\limits_{j=1}^K \frac{1}{n_j} \sum\limits_{i=1}^{n_j} (x_i - \mu_j) \cdot (x_i - \mu_j)^T
$$

In [ ]:
def NC1_byhand(X, y):
    """ NC computation without using the covariance operator """
    # Deprecated version which can be ignored
    nclasses = len(np.unique(y))
    ohe_y = np.eye(nclasses)[y]
    n_per_class = np.sum(ohe_y, axis=0)
    sum_per_class = (X.T @ ohe_y)
    mean_per_class = sum_per_class / n_per_class
    X_class_norm = X - ohe_y @ mean_per_class.T
    n_per_class_examples = ohe_y @ n_per_class
    X_class_norm_balanced = X_class_norm / np.sqrt(n_per_class_examples)[:, None]
    SigmaW = (1.0/nclasses) * X_class_norm_balanced.T @ X_class_norm_balanced
    
    
    mean = np.mean(X, axis=0)
    mean_per_class_norm = mean_per_class - mean[:, None] 
    SigmaB = (1.0/nclasses) * mean_per_class_norm @ mean_per_class_norm.T 

    NC_value = np.trace(SigmaW @ pinv(SigmaB))/nclasses

    return NC_value, nclasses


def NC1(X, y, 
        inversion_method="pinv", 
        return_matrices=False,
        normalization="overall"):
    """ Faster version using the covariance operator 
    @param inversion_method "pinv", "eig"
    @param normalization "overall" or "class" 
    """
    classes = np.unique(y)
    nclasses = len(classes)
    dim = X.shape[1]
    classmeans = []
    SigmaW = np.zeros((dim, dim))
    for i in classes:
        class_examples = X[y==i, :].T
        C = np.cov(class_examples, bias=True)
        if normalization=="class":
            SigmaW += C
        else:
            SigmaW += C * class_examples.shape[1]
        classmean = np.mean(class_examples, axis=1)
        classmeans.append(classmean)

    if normalization=="class":
        SigmaW /= nclasses
    else:
        SigmaW /= X.shape[0]
    
    classmeans = np.array(classmeans).T
    SigmaB = np.cov(classmeans, bias=True).reshape(dim,dim) # reshaping for the one-dimensional case

    if inversion_method=="pinv":
        NC_value = np.trace(SigmaW @ pinv(SigmaB)) / nclasses
    elif inversion_method=="eig":
        e, eigv = np.linalg.eig(SigmaB)
        # this is ignored in https://github.com/rhubarbwu/neural-collapse/blob/main/neural_collapse/measure.py (L48)
        nonzero_e = np.abs(e)>1e-8 
        e[nonzero_e] = 1.0 / e[nonzero_e]
        NC_value = np.real(np.trace(SigmaW @ eigv @ np.diag(e) @ eigv.T) / nclasses)

    if return_matrices:
        return NC_value, nclasses, SigmaB, SigmaW
    else:
        return NC_value, nclasses


def MeanDistanceRatio(X, y):
    """ Blue lines of Fig. 2 in https://www.pnas.org/doi/full/10.1073/pnas.2015509117 """
    classes = np.unique(y)
    nclasses = len(classes)
    dim = X.shape[1]
    mean_distances = []
    global_mean = np.mean(X)
    for i in classes:
        class_examples = X[y==i, :].T
        classmean = np.mean(class_examples, axis=1)
        mean_distances.append(np.linalg.norm(classmean-global_mean))

    return np.std(mean_distances)/np.mean(mean_distances), nclasses

In [ ]:
NCMeasure = MeanDistanceRatio

## Analyzing all scikit learn datasets for classification

In [ ]:
data = [ lambda: datasets.load_iris(),
         lambda: datasets.load_digits(),
         lambda: datasets.load_wine(),
         lambda: datasets.load_breast_cancer() ]

for dataset in data:
    D = dataset()
    X, y = D.data, D.target
    NC_value1, nclasses = NCMeasure(X, y)

    print (f"{nclasses:5d} {NC_value1:8.3f}")

## Sample classes from the digits dataset

In [ ]:
D = datasets.load_digits()
X, y = D.data, D.target
class_idx = np.unique(y)
max_samples = 20
rng = np.random.default_rng()
class_range = range(2, 10)
NC = {}
for c in class_range:
    NC_cs = []
    for i in range(max_samples):
        s = rng.choice(class_idx, size=c, replace=False, shuffle=True)
        X_s = X[np.in1d(y, s)]
        y_s = y[np.in1d(y, s)]
        NC_c, _ = NCMeasure(X_s, y_s)
        NC_cs.append(NC_c)
        
    NC[c] = NC_cs        

In [ ]:
plt.errorbar(class_range, 
             [ np.mean(NC[c]) for c in class_range ],
             [ np.std(NC[c]) for c in class_range ],
            )
plt.xlabel("number of classes")
plt.ylabel("NC")

## Experiments with synthetic data

In [ ]:
def make_gaussian_classification(n_samples, n_features, n_classes, n_clusters_per_class):
    # choose global mean
    # 1. mu sampled from N(0,I)
    global_mean = np.random.randn(n_features)
    global_cov = np.eye(n_features)
    X = []
    y = []
    # 2. mu_j sampled from N(mu, I), i.e. from N(0, I)
    class_means = np.random.multivariate_normal(global_mean, global_cov, size=n_classes)
    for i in range(n_classes):
        class_mean = class_means[i,:]
        # 3. cluster_{j,l} sampled from N(mu_j, 0.1*I)
        cluster_means = np.random.multivariate_normal(class_mean, global_cov*0.1, size=n_clusters_per_class)
        for k in range(n_clusters_per_class):
            cluster_mean = cluster_means[k,:]
            n_examples = n_samples // (n_classes * n_clusters_per_class)
            # 4. x sampled from N(cluster_{j,l}, 0.01*I)
            examples = np.random.multivariate_normal(cluster_mean, global_cov*0.01, size=n_examples)
            X.append(examples)
            y.append(np.ones(n_examples)*i)

    return np.vstack(X), np.hstack(y).astype(int)
            

In [ ]:
X, y = make_gaussian_classification(n_samples=128, n_features=2, n_classes=4, n_clusters_per_class=2)
plt.scatter(X[:,0], X[:,1], c=y)

In [ ]:
D = 32
nclasses_range = np.arange(2,2*D)

generation_methods = {"sklearn": lambda n_classes: datasets.make_classification(
                                        n_samples=1024, 
                                        n_features=D, 
                                        n_informative=D, 
                                        n_redundant=0, 
                                        n_repeated=0, 
                                        n_classes=n_classes, 
                                        n_clusters_per_class=1),
                      "gaussian": lambda n_classes: make_gaussian_classification(n_samples=1024, 
                                                                   n_features=D, 
                                                                   n_classes=n_classes, 
                                                                   n_clusters_per_class=1)}

generation_method = generation_methods["gaussian"]

NC_values = []
nclasses_values = [] 

for k in nclasses_range:
    X, y = generation_method(k)
    NC, nclasses = NCMeasure(X, y)
    NC_values.append(NC)
    nclasses_values.append(nclasses)

nclasses_values = np.array(nclasses_values)

In [ ]:
plt.plot(nclasses_values, NC_values)
plt.xlabel("Number of synthetic classes")
plt.ylabel("NC value")